In [ ]:
import openai
import json

# Utility Function
Used to read the Open AI API from your apikey file.

In [3]:
import os 
import yaml
def read_config(path):
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The API key for the Amadeus Flights API.
    """
    
    # Get the directory of the current script
    script_dir = path

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        configs = yaml.safe_load(stream)
        API_KEY = configs['openai']['api_key']
            
    return API_KEY

In [4]:
path = r"C:\Users\johna\OneDrive\Documents\api_keys"  # Change to the location of your apikeys.yml
API_KEY = read_config(path)

# Setting up our functions
Let'smimagine we wanted to put a natural language interface on top of an API.
We can mimic an API with these timple functions. One to give us news headlines, and another to return the price of financial assets.

In [46]:
import openai

def generate_response(topic, API_KEY):

    """
    Generates a random news headline for a given ticker
    """
    
    openai.api_key = API_KEY
    
    content = "You generate news headlines"
    prompt = f"generate a random news headline based on the f{topic}"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        n=1,
        stop=None,
        temperature=0.5,
    )

    return response.choices[0].message['content'].strip()


def get_current_news(topic):
    """Get the latest news on a given topic"""
    headline = generate_response(topic, API_KEY)
    
    news_round = {
        "topic": topic,
        "headline": headline
    }
    return json.dumps(news_round)

In [66]:
import pandas as pd
import numpy as np
def date_to_number(date):
    """
    Converts a date to a number.

    Parameters:
    date (str): The date in 'YYYY-MM-DD' format.

    Returns:
    float: The number representation of the date.
    """
    timestamp = pd.to_datetime(date).timestamp()
    number = np.round(np.log(np.tan(timestamp)), 2)
    return number

def get_current_price(ticker, date, currency):
    """
    Get the price for a given ticker, 
    at a given date and in a given currency
    """

    price = date_to_number(date)
    news_round = {
        "ticker": ticker,
        "price": price,
        "date": date,
        "currency": currency
    }
    return json.dumps(news_round)

# Function Calling with One Function
Example of how function calling works with just one function

In [10]:
import openai

def function_calling_one_function(API_KEY, query):
    # Step 1: send the conversation and available functions to GPT
    functions = [
        {
            "name": "get_ticker_price",
            "description": "Get the ticker price of at a given date in a given currency",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Return the ticker symbol for the compnay mentioned. E.g Microsoft Corporation as MSFT."
                    },
                    "date": {
                        "type": "string",
                        "description": "Return the date in in 'YYYY-MM-DD' format."
                    },
                    "currency": {
                        "type": "string",
                        "description": "return the curency in use the ISO 4217 code. E.g. United States Dollar as USD"
                    }
                 },
                },
                "required": ["ticker", "date", "currency"],
            }
    ]

    openai.api_key = API_KEY
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "system", "content": query}],
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    return response_message



In [11]:
import openai

def function_calling_multiple_function(API_KEY, query):
    # Step 1: send the conversation and available functions to GPT
    functions = [
        {
            "name": "get_current_news",
            "description": "Get the latest news headline for a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The news topic required "
                    }
                 },
                },
                "required": ["topic"],
            },

        {
            "name": "get_ticker_price",
            "description": "Get the ticker price of at a given date in a given currency",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Return the ticker symbol for the compnay mentioned. E.g Microsoft Corporation as MSFT."
                    },
                    "date": {
                        "type": "string",
                        "description": "Return the date in in 'YYYY-MM-DD' format."
                    },
                    "currency": {
                        "type": "string",
                        "description": "return the curency in use the ISO 4217 code. E.g. United States Dollar as USD"
                    }
                 },
                },
                "required": ["ticker", "date", "currency"],
            }
    ]

    openai.api_key = API_KEY
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "system", "content": query}],
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    return response_message

In [ ]:
    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

In [12]:
query = "Give me the price of bitcoin on the 1st of march 2023 in pounds"
function_calling_one_function(API_KEY, query)

<OpenAIObject at 0x2b62993e2c0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_ticker_price",
    "arguments": "{\n  \"ticker\": \"BTC\",\n  \"date\": \"2023-03-01\",\n  \"currency\": \"GBP\"\n}"
  }
}

In [29]:
query_1 = "Give me the price of bitcoin on the 1st of march 2023 in pounds"
query_2 = "Give me the latest headlines on forest fires in greece"
query_3 = "what's 10 + 10?"

In [30]:
query_1_response = function_calling_multiple_function(API_KEY, query_1)
query_1_response

<OpenAIObject at 0x2b629baab30> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_ticker_price",
    "arguments": "{\n\"ticker\": \"BTC\",\n\"date\": \"2023-03-01\",\n\"currency\": \"GBP\"\n}"
  }
}

In [31]:
query_2_response = function_calling_multiple_function(API_KEY, query_2)
query_2_response

<OpenAIObject at 0x2b629ba74f0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_news",
    "arguments": "{\n  \"topic\": \"forest fires in greece\"\n}"
  }
}

In [18]:
query_3 = function_calling_multiple_function(API_KEY, query_3)

<OpenAIObject at 0x2b62993ee50> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_news",
    "arguments": "{}"
  }
}

# What happens if you try to do this without function calling?
Responses are hit and miss, sometimes doesn't respond to exact specifications. No additional flexibility.

In [24]:
def return_news_args(query, API_KEY):

    """
    Generates a random news headline for a given ticker
    """
    
    openai.api_key = API_KEY
    
    content = "You parse the arguments defined in the prompt"
    prompt = f""" From the {query} return the following. ticker: "Return the ticker symbol for the compnay mentioned. E.g Microsoft Corporation as MSFT."
                    date: "Return the date in in 'YYYY-MM-DD' format."
                    currency : return the curency in use the ISO 4217 code. E.g. United States Dollar as USD """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        n=1,
        stop=None,
        temperature=0,
    )

    return response.choices[0].message['content'].strip()

In [26]:
return_news_args(query_1, API_KEY)

'Based on the given prompt, the arguments to parse are as follows:\n\n1. Ticker Symbol: "bitcoin"\n2. Date: "1st of March 2023"\n3. Currency: "pounds"'

# Let's parse the function call outputs and feed them back into the function

In [39]:
import json
def parse_function(response):
    parsed_output = response["function_call"]["arguments"]
    parsed_output = json.loads(parsed_output)
    return parsed_output

In [41]:
response = query_1_response
ticker = parse_function(response)['ticker']
date = parse_function(response)['date']
currency = parse_function(response)['currency']

In [42]:
response = query_2_response
topic = parse_function(response)['topic']

In [48]:
get_current_news(topic)

'{"topic": "forest fires in greece", "headline": "\\"Greece Battles Devastating Forest Fires: Efforts Intensify to Control the Blaze and Protect Lives\\""}'

In [67]:
get_current_price(ticker, date, currency)

'{"ticker": "BTC", "price": 0.99, "date": "2023-03-01", "currency": "GBP"}'

In [ ]:
def create_story(query, API_KEY):

    """
    Generates a random news headline for a given ticker
    """
    
    openai.api_key = API_KEY
    
    content = "You parse the arguments defined in the prompt"
    prompt = f""" From the {query} return the following. ticker: "Return the ticker symbol for the compnay mentioned. E.g Microsoft Corporation as MSFT."
                    date: "Return the date in in 'YYYY-MM-DD' format."
                    currency : return the curency in use the ISO 4217 code. E.g. United States Dollar as USD """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        n=1,
        stop=None,
        temperature=0,
    )

    return response.choices[0].message['content'].strip()